In [1]:
from scipy.optimize import linprog

# Змінні:
# x[0] — темп стрільби зброї A (залпів за хвилину)
# x[1] — темп стрільби зброї B (залпів за хвилину)
# x[2] — темп стрільби зброї C (залпів за хвилину)

# Цільова функція — максимізувати загальну кількість залпів:
# Зброя A: 3 хв * x[0], B: 5 хв * x[1], C: 4 хв * x[2] — треба мінімізувати від'ємне значення
c = [-3, -5, -4]

# Матриця обмежень (ліва частина A_ub * x <= b_ub)
A = [
    [3, 2, 4],    # Загальна кількість залпів у межах 15
    [2, 3, 0],    # Обмеження на інший період ≤ 8
    [1, 0, -1]    # Різниця A (3 хв) та C (1 хв) ≤ 5
]

b = [
    15,  # загальна кількість залпів
    8,   # додаткове обмеження
    5    # різниця у вогневій потужності
]

# Межі змінних (усі значення мають бути ≥ 0)
x_bounds = [(0, None), (0, None), (0, None)]

# Виконання оптимізації
res = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Виведення результатів
if res.success:
    print("Оптимальний темп стрільби:")
    print(f"Зброя A: {res.x[0]:.2f} залпів/хв")
    print(f"Зброя B: {res.x[1]:.2f} залпів/хв")
    print(f"Зброя C: {res.x[2]:.2f} залпів/хв")
    print(f"Максимальна загальна кількість залпів: {-res.fun:.2f}")
else:
    print("Рішення не знайдено:", res.message)


Оптимальний темп стрільби:
Зброя A: 0.00 залпів/хв
Зброя B: 2.67 залпів/хв
Зброя C: 2.42 залпів/хв
Максимальна загальна кількість залпів: 23.00


In [1]:
import numpy as np
from scipy.optimize import linprog

# Витрати на перевезення (матриця 5x5 — постачальники x споживачі)
costs = np.array([
    [2, 4, 3, 5, 2],
    [3, 1, 4, 2, 2],
    [5, 3, 2, 6, 8],
    [1, 4, 3, 3, 4],
    [5, 1, 1, 5, 3]
])

# Пропозиція від постачальників
supply = np.array([40, 85, 120, 15, 100])

# Попит споживачів
demand = np.array([20, 80, 80, 80, 100])

# Перевіримо баланс задачі:
if supply.sum() != demand.sum():
    print("Задача незбалансована, додамо фіктивного постачальника або споживача.")
    if supply.sum() > demand.sum():
        # Додаємо фіктивного споживача
        costs = np.column_stack((costs, np.zeros(len(supply))))
        demand = np.append(demand, supply.sum() - demand.sum())
    else:
        # Додаємо фіктивного постачальника
        costs = np.vstack((costs, np.zeros(len(demand))))
        supply = np.append(supply, demand.sum() - supply.sum())

# Задаємо цільову функцію (лінійно, у вектор)
c = costs.flatten()

# Обмеження — запаси постачальників (<=)
A_ub = []
b_ub = []
for i in range(len(supply)):
    constraint = np.zeros(costs.size)
    constraint[i*len(demand):(i+1)*len(demand)] = 1
    A_ub.append(constraint)
    b_ub.append(supply[i])

# Обмеження — потреби споживачів (>=)
A_eq = []
b_eq = []
for j in range(len(demand)):
    constraint = np.zeros(costs.size)
    constraint[j::len(demand)] = 1
    A_eq.append(constraint)
    b_eq.append(demand[j])

# Межі змінних (всі >= 0)
bounds = [(0, None) for _ in range(costs.size)]

# Розв'язання задачі
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Вивід результату
if res.success:
    print("Оптимальні обсяги перевезень:")
    print(res.x.reshape(costs.shape))
    print(f"Загальні витрати: {res.fun}")
else:
    print("Задача не має розв'язку")



Оптимальні обсяги перевезень:
[[ 0.  0.  0.  0. 40.]
 [ 0.  0.  0. 80.  5.]
 [ 5. 35. 80.  0.  0.]
 [15.  0.  0.  0.  0.]
 [ 0. 45.  0.  0. 55.]]
Загальні витрати: 765.0


In [2]:
import numpy as np
from scipy.optimize import linprog

# Матриця витрат (5 постачальників x 5 споживачів)
costs = np.array([
    [2, 4, 3, 5, 2],
    [3, 1, 4, 2, 2],
    [5, 3, 2, 6, 8],
    [1, 4, 3, 3, 4],
    [5, 1, 1, 5, 3]
])

# Пропозиції постачальників
supply = np.array([40, 85, 120, 15, 100])

# Попит споживачів
demand = np.array([20, 80, 80, 80, 100])

# Балансування задачі
if supply.sum() != demand.sum():
    print("Задача незбалансована, додаємо фіктивного постачальника або споживача.")
    if supply.sum() > demand.sum():
        costs = np.column_stack((costs, np.zeros(len(supply))))
        demand = np.append(demand, supply.sum() - demand.sum())
    else:
        costs = np.vstack((costs, np.zeros(len(demand))))
        supply = np.append(supply, demand.sum() - supply.sum())

# Цільова функція (витрати — розгорнута вектором)
c = costs.flatten()

# Обмеження за запасами постачальників (<=)
A_ub = []
b_ub = []
for i in range(len(supply)):
    constraint = np.zeros(costs.size)
    constraint[i*len(demand):(i+1)*len(demand)] = 1
    A_ub.append(constraint)
    b_ub.append(supply[i])

# Обмеження за попитом споживачів (>= — але перетворимо на рівність і невід'ємність)
A_eq = []
b_eq = []
for j in range(len(demand)):
    constraint = np.zeros(costs.size)
    constraint[j::len(demand)] = 1
    A_eq.append(constraint)
    b_eq.append(demand[j])

# Межі змінних (усі >= 0)
bounds = [(0, None) for _ in range(costs.size)]

# Розв'язок симплекс-методом
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')

# Виведення результату
if res.success:
    print("Оптимальні обсяги перевезень (матриця):")
    print(res.x.reshape(costs.shape))
    print(f"Загальні витрати: {res.fun}")
else:
    print("Задача не має розв'язку")



Оптимальні обсяги перевезень (матриця):
[[ 0.  0.  0.  0. 40.]
 [ 0.  0.  0. 80.  5.]
 [ 5. 35. 80.  0.  0.]
 [15.  0.  0.  0.  0.]
 [ 0. 45.  0.  0. 55.]]
Загальні витрати: 765.0


C:\Users\TZ\AppData\Local\Temp\ipykernel_15924\4183862680.py:54: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')


In [4]:
import numpy as np
from scipy.optimize import linprog

costs = np.array([
    [2, 4, 3, 5, 2],
    [3, 1, 4, 2, 2],
    [5, 3, 2, 6, 8],
    [1, 4, 3, 3, 4],
    [5, 1, 1, 5, 3]
])

supply = np.array([40, 85, 120, 15, 100])
demand = np.array([20, 80, 80, 80, 100])

# Перевіряємо баланс — тепер залишаємо тільки перевірку без додавання нічого
assert supply.sum() == demand.sum(), "Задача незбалансована!"

c = costs.flatten()

# Обмеження постачальників (<=)
A_ub = []
b_ub = []
for i in range(len(supply)):
    constraint = np.zeros(costs.size)
    constraint[i*len(demand):(i+1)*len(demand)] = 1
    A_ub.append(constraint)
    b_ub.append(supply[i])

# Обмеження споживачів (=)
A_eq = []
b_eq = []
for j in range(len(demand)):
    constraint = np.zeros(costs.size)
    constraint[j::len(demand)] = 1
    A_eq.append(constraint)
    b_eq.append(demand[j])

bounds = [(0, None) for _ in range(costs.size)]

# Симплекс-метод
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')

if res.success:
    print("Оптимальні обсяги перевезень:")
    print(res.x.reshape(costs.shape))
    print(f"Загальні витрати: {res.fun}")
else:
    print("Задача не має розв'язку")


C:\Users\TZ\AppData\Local\Temp\ipykernel_15924\3036367708.py:41: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')


Оптимальні обсяги перевезень:
[[ 0.  0.  0.  0. 40.]
 [ 0.  0.  0. 80.  5.]
 [ 5. 35. 80.  0.  0.]
 [15.  0.  0.  0.  0.]
 [ 0. 45.  0.  0. 55.]]
Загальні витрати: 765.0


In [5]:
import numpy as np
from scipy.optimize import linprog

costs = np.array([
    [2, 4, 3, 5, 2],
    [3, 1, 4, 2, 2],
    [5, 3, 2, 6, 8],
    [1, 4, 3, 3, 4],
    [5, 1, 1, 5, 3]
])

supply = np.array([40, 85, 120, 15, 100])
demand = np.array([20, 80, 80, 80, 100])

# Перевіряємо баланс — тепер залишаємо тільки перевірку без додавання нічого
assert supply.sum() == demand.sum(), "Задача незбалансована!"

c = costs.flatten()

# Обмеження постачальників (<=)
A_ub = []
b_ub = []
for i in range(len(supply)):
    constraint = np.zeros(costs.size)
    constraint[i*len(demand):(i+1)*len(demand)] = 1
    A_ub.append(constraint)
    b_ub.append(supply[i])

# Обмеження споживачів (=)
A_eq = []
b_eq = []
for j in range(len(demand)):
    constraint = np.zeros(costs.size)
    constraint[j::len(demand)] = 1
    A_eq.append(constraint)
    b_eq.append(demand[j])

bounds = [(0, None) for _ in range(costs.size)]

# Симплекс-метод
res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')

if res.success:
    print("Оптимальні обсяги перевезень:")
    print(res.x.reshape(costs.shape))
    print(f"Загальні витрати: {res.fun}")
else:
    print("Задача не має розв'язку")


Оптимальні обсяги перевезень:
[[ 0.  0.  0.  0. 40.]
 [ 0.  0.  0. 80.  5.]
 [ 5. 35. 80.  0.  0.]
 [15.  0.  0.  0.  0.]
 [ 0. 45.  0.  0. 55.]]
Загальні витрати: 765.0


C:\Users\TZ\AppData\Local\Temp\ipykernel_15924\3036367708.py:41: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
